In [1]:
import torch
import torch.nn as nn
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch.optim as optim
import torch.nn.functional as F
# from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import models
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
# import time
import cv2
import os


In [2]:
for dirname, _, filenames in os.walk('/kaggle/input/cifar10-python'):
    for foldername in _:
        print(os.path.join(dirname, foldername))
        break

/kaggle/input/cifar10-python/cifar-10-batches-py


In [3]:
# transform = transforms.Compose([
# #     [transforms.RandomResizedCrop(255),
# #      transforms.CenterCrop(224),  
#      transforms.Resize(size=(150 , 150)) ,
#      transforms.ColorJitter(0.4,0.5,0.5,0.2),
#      transforms.RandomHorizontalFlip(p=0.5) , 
#      transforms.RandomCrop(size=(150,150)),  
    
#      transforms.ToTensor(),
#      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

train_transforms = transforms.Compose([
    transforms.Resize(size=(150 , 150)) ,
     transforms.ColorJitter(0.4,0.5,0.5,0.2),
     transforms.RandomHorizontalFlip(p=0.4) , 
     transforms.RandomCrop(size=(150,150)),     
     transforms.ToTensor(),
     transforms.Normalize((0.425, 0.415, 0.405), (0.205, 0.205, 0.205))])


test_transforms = transforms.Compose([
    transforms.Resize((150, 150)),      
    transforms.ToTensor(),
    transforms.Normalize((0.425, 0.415, 0.405), (0.255, 0.245, 0.235))
])


trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transforms)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_transforms)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)


100%|██████████| 170498071/170498071 [00:01<00:00, 99712533.55it/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
train_len=len(trainset)
test_len=len(testset)

In [5]:
print(test_len)

10000


In [6]:
print(len(trainset.classes))

trainset.classes
# same classes in test data 

10


['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [7]:
model = models.resnet50(pretrained=True)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 164MB/s] 


In [8]:
# for param in model.parameters(): # freezes the layers
#     param.required_grad = False
# number_feature = model.fc.in_features
# model.fc = torch.nn.Linear(in_features=number_feature , out_features=10, bias=True)

In [9]:
# Freeze all layers except the final classification layer
for name, param in model.named_parameters():
    if "fc" in name:  # Unfreeze the final classification layer
        param.requires_grad = True
    else:
        param.requires_grad = False


In [10]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device is ', device)

device is  cuda


In [11]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
schedul_learning = torch.optim.lr_scheduler.MultiStepLR(optimizer=optimizer , milestones=[3 , 6 ] ,
                                                        gamma=0.5)

In [14]:
model.train()
    
for epoch in range(20):
    total_correct = 0.0 
    running_loss = 0.0 
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.cuda(), labels.cuda()
        # Forward pass
        output = model(inputs)
        # Get predicted classes
        output_idex = torch.argmax(output, dim=1)
        # Calculate the number of correct predictions
        total_correct += (labels == output_idex).sum().item()
        # Zero the parameter gradients
        optimizer.zero_grad()
        # Calculate loss
        loss = criterion(output, labels)  # Fixed typo: 'cirterion' to 'criterion'
        running_loss += loss.item() * inputs.size(0)
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
    print('epoch: ', epoch, 'loss: ', running_loss / train_len, 'accuracy: ', (total_correct / train_len) * 100, '%')

    if (epoch + 1) % 3 == 0:
            Data_Snapshot = f'/kaggle/working/cnnnet_epoch_{epoch+1}.pth'
            torch.save(model.state_dict(), Data_Snapshot)
            print(f" saved at {Data_Snapshot}")
    print("---------------------------")

print('finished training')


epoch:  0 loss:  1.0146068847465515 accuracy:  66.704 %
---------------------------
epoch:  1 loss:  0.7963647353076935 accuracy:  72.76599999999999 %
---------------------------
epoch:  2 loss:  0.7743401646614074 accuracy:  73.76 %
 saved at /kaggle/working/cnnnet_epoch_3.pth
---------------------------
epoch:  3 loss:  0.7466917690753937 accuracy:  74.276 %
---------------------------
epoch:  5 loss:  0.7228994320201874 accuracy:  75.234 %
 saved at /kaggle/working/cnnnet_epoch_6.pth
---------------------------
epoch:  6 loss:  0.7195748014545441 accuracy:  75.468 %
---------------------------
epoch:  7 loss:  0.7127152788257599 accuracy:  75.554 %
---------------------------
epoch:  8 loss:  0.7048812763404846 accuracy:  75.928 %
 saved at /kaggle/working/cnnnet_epoch_9.pth
---------------------------
epoch:  9 loss:  0.6917364977455139 accuracy:  76.266 %
---------------------------
epoch:  10 loss:  0.6951166493034363 accuracy:  76.294 %
---------------------------
epoch:  11 los

In [15]:
# Test

with torch.no_grad():
    model.eval()
    total_loss=0.0 
    total_correct=0.0 
    for inputs, labels in testloader:
        labels=labels.to(device)
        outputs=model(inputs.to(device))
        loss=criterion(outputs, labels)
        total_loss+=loss.item()*inputs.size(0)
        output_idx=torch.argmax(outputs, dim=1)
        total_correct+=sum(labels==output_idx).sum().item() 
        accuracy = (total_correct / test_len) * 100
        loss = total_loss / test_len

    print(f'Accuracy: {accuracy:.2f}% Loss: {loss:.4f}')

Accuracy: 82.06% Loss: 0.5240
